<a href="https://colab.research.google.com/github/Valerik-Bogd/Punctuation/blob/main/Rubert_LSTM_punct_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Импорты

In [ ]:
# # common_path = '/content/drive/MyDrive/Colab Notebooks/punctuation_23_10_11/debug_data'
# # common_save_path = '/content/drive/MyDrive/Colab Notebooks/punctuation_23_10_11/'

# common_path = '/content/drive/MyDrive/Colab Notebooks/punctuation_23_10_11/datasets/ready-to-use'
# common_save_path = common_path

common_path = '/content/drive/MyDrive/Data_punct/Datasets'
common_save_path = common_path

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers
!pip install labml

from contextlib import ExitStack
from pathlib import Path
from typing import Tuple

import numpy as np
import torch
import torch.multiprocessing
import torch.nn as nn
import argparse
from labml import experiment, tracker
from numpy import ndarray
from torch.utils.data import DataLoader
from tqdm import tqdm
import os
from pathlib import Path
from typing import Dict, List, Optional, Tuple, TypedDict, Union
from torch import Tensor
from transformers import PreTrainedModel
from transformers import PreTrainedTokenizer
from transformers import AutoModel, AutoTokenizer
from transformers import AutoModel, AutoTokenizer


from google.colab import drive
import sys
from IPython.display import clear_output

clear_output()

In [ ]:
# https://github.com/pytorch/pytorch/issues/11201
torch.multiprocessing.set_sharing_strategy('file_system')

# for reproducibility
seed = 42
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(seed)

In [ ]:
from google.colab import drive
from pathlib import Path
drive.mount('/content/drive/')
sys.path.insert(0, common_path)

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
!ls

drive  sample_data


In [ ]:
PRETRAINED_MODELS = {
      'bert-base-uncased': (768, 'bert'),
      'bert-large-uncased': (1024, 'bert'),
      'bert-base-multilingual-cased': (768, 'bert'),
      'bert-base-multilingual-uncased': (768, 'bert'),
      # 'xlm-mlm-en-2048': (2048, 'xlm'),
      # 'xlm-mlm-100-1280': (1280, 'xlm'),
      'roberta-base': (768, 'roberta'),
      'roberta-large': (1024, 'roberta'),
      'distilbert-base-uncased': (768, 'bert'),
      'distilbert-base-multilingual-cased': (768, 'bert'),
      'xlm-roberta-base': (768, 'roberta'),
      'xlm-roberta-large': (1024, 'roberta'),
      'albert-base-v1': (768, 'albert'),
      'albert-base-v2': (768, 'albert'),
      'albert-large-v2': (1024, 'albert'),
      'DeepPavlov/rubert-base-cased-sentence': (768, 'bert'),
}

In [ ]:
TOKEN_IDX = {}
targets = {}

def get_special_tokens(name, tokenizer):
    names_special = tokenizer.all_special_tokens
    special_ids = tokenizer.all_special_ids
    return {name: dict(zip(names_special, special_ids))}


def get_punct_tokens(name, tokenizer):
    all_punct = ["'", '"',
                 ".", ",", "!", "?",
                 "-", ":", ";"]
    names_punct = [
                   'S_QUOTE', 'D_QUOTE',
                   'PERIOD', 'COMMA', 'EX_MARK', 'Q_MARK',
                   'DASH', 'COLUMN', 'SEMI_COL'
                   ]

    tokenized_data = tokenizer(all_punct)

    tokens_dict = {}
    for one_mark, tokens in zip(names_punct, tokenized_data["input_ids"]):
        if len(tokens) > 3:
          continue
        tokens_dict[one_mark] = tokens[1]
    return {name: tokens_dict}

# space is usually [SEP] special token and shouldn't be mask 1

In [ ]:
for name_of_model in PRETRAINED_MODELS.keys():
    print(name_of_model)
    model = AutoModel.from_pretrained(name_of_model)
    tokenizer = AutoTokenizer.from_pretrained(name_of_model)
    TOKEN_IDX.update(get_special_tokens(name=name_of_model, tokenizer=tokenizer))
    targets.update(get_punct_tokens(name=name_of_model, tokenizer=tokenizer))
    del model
    clear_output()

In [ ]:
print('TOKEN_IDX')
for k, v in TOKEN_IDX.items():
    print(k, v)
print('-----------')
print()
print('targets')
for k, v in targets.items():
    print(k, v)

TOKEN_IDX
bert-base-uncased {'[UNK]': 100, '[SEP]': 102, '[PAD]': 0, '[CLS]': 101, '[MASK]': 103}
bert-large-uncased {'[UNK]': 100, '[SEP]': 102, '[PAD]': 0, '[CLS]': 101, '[MASK]': 103}
bert-base-multilingual-cased {'[UNK]': 100, '[SEP]': 102, '[PAD]': 0, '[CLS]': 101, '[MASK]': 103}
bert-base-multilingual-uncased {'[UNK]': 100, '[SEP]': 102, '[PAD]': 0, '[CLS]': 101, '[MASK]': 103}
roberta-base {'<s>': 0, '</s>': 2, '<unk>': 3, '<pad>': 1, '<mask>': 50264}
roberta-large {'<s>': 0, '</s>': 2, '<unk>': 3, '<pad>': 1, '<mask>': 50264}
distilbert-base-uncased {'[UNK]': 100, '[SEP]': 102, '[PAD]': 0, '[CLS]': 101, '[MASK]': 103}
distilbert-base-multilingual-cased {'[UNK]': 100, '[SEP]': 102, '[PAD]': 0, '[CLS]': 101, '[MASK]': 103}
xlm-roberta-base {'<s>': 0, '</s>': 2, '<unk>': 3, '<pad>': 1, '<mask>': 250001}
xlm-roberta-large {'<s>': 0, '</s>': 2, '<unk>': 3, '<pad>': 1, '<mask>': 250001}
albert-base-v1 {'[CLS]': 2, '[SEP]': 3, '<unk>': 1, '<pad>': 0, '[MASK]': 4}
albert-base-v2 {'[CLS


# Служебное

In [ ]:
model = AutoModel.from_pretrained('DeepPavlov/rubert-base-cased-sentence') #####################################
tokenizer = AutoTokenizer.from_pretrained('DeepPavlov/rubert-base-cased-sentence') ##################################

config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/711M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
from typing import Dict, List, Optional, Tuple, TypedDict, Union

import numpy as np
import torch
from torch import Tensor
from tqdm import tqdm
from transformers import PreTrainedTokenizer

# probability of applying substitution operation on tokens selected for augmentation
alpha_sub = 0.40
# probability of applying delete operation on tokens selected for augmentation
alpha_del = 0.40

tokenizer = None
# substitution strategy: 'unk' -> replace with unknown tokens, 'rand' -> replace with random tokens from vocabulary
sub_style = 'unk'


def augment_none(x, y, y_mask, x_aug, y_aug, y_mask_aug, i, token_style):
    """
    apply no augmentation
    """
    x_aug.append(x[i])
    y_aug.append(y[i])
    y_mask_aug.append(y_mask[i])


def augment_substitute(x, y, y_mask, x_aug, y_aug, y_mask_aug, i, token_style):
    """
    replace a token with a random token or the unknown token
    """
    if sub_style == 'rand':
        x_aug.append(np.random.randint(tokenizer.vocab_size))
    else:
        x_aug.append(TOKEN_IDX[token_style]['UNK'])
    y_aug.append(y[i])
    y_mask_aug.append(y_mask[i])


def augment_insert(x, y, y_mask, x_aug, y_aug, y_mask_aug, i, token_style):
    """
    insert the unknown token before this token
    """
    x_aug.append(TOKEN_IDX[token_style]['UNK'])
    y_aug.append(0)
    y_mask_aug.append(1)
    x_aug.append(x[i])
    y_aug.append(y[i])
    y_mask_aug.append(y_mask[i])


def augment_delete(x, y, y_mask, x_aug, y_aug, y_mask_aug, i, token_style):
    """
    remove this token i..e, not add in augmented tokens
    """
    return


def augment_all(x, y, y_mask, x_aug, y_aug, y_mask_aug, i, token_style):
    """
    apply substitution with alpha_sub probability, deletion with alpha_sub probability and insertion with
    1-(alpha_sub+alpha_sub) probability
    """
    r = np.random.rand()
    if r < alpha_sub:
        augment_substitute(x, y, y_mask, x_aug, y_aug, y_mask_aug, i, token_style)
    elif r < alpha_sub + alpha_del:
        augment_delete(x, y, y_mask, x_aug, y_aug, y_mask_aug, i, token_style)
    else:
        augment_insert(x, y, y_mask, x_aug, y_aug, y_mask_aug, i, token_style)

AUGMENTATIONS = {'none': augment_none,
    'substitute': augment_substitute,
    'insert': augment_insert,
    'delete': augment_delete,
    'all': augment_all}

TOKEN_IDX = {
      'bert': {
          # '[MASK]': 103,
          'S_QUOTE': 118,
          'D_QUOTE': 108,
          'PERIOD': 132,
          'COMMA': 128,
          'EX_MARK': 106,
          'Q_MARK': 166,
          'DASH': 130,
          'COLUMN': 156,
          'SEMI_COL': 158,
          'START_SEQ': 101,
          'PAD': 0,
          'END_SEQ': 102,
          'UNK': 100
      },
      'xlm': {
          'START_SEQ': 0,
          'PAD': 2,
          'END_SEQ': 1,
          'UNK': 3
      },
      'roberta': {
          'START_SEQ': 0,
          'PAD': 1,
          'END_SEQ': 2,
          'UNK': 3
      },
      'albert': {
          'START_SEQ': 2,
          'PAD': 0,
          'END_SEQ': 3,
          'UNK': 1
      },
  }

In [ ]:
PRETRAINED_MODELS = {'DeepPavlov/rubert-base-cased-sentence': (AutoModel, AutoTokenizer, 768, 'bert')}

Path_type = Union[Path, str, os.PathLike]


class CorrectionModel(nn.Module):
    def __init__(self,
                 pretrained_model: PreTrainedModel,
                 targets: Dict[str, int],
                 freeze_pretrained=False,
                 lstm_dim=-1) -> None:

        super(CorrectionModel, self).__init__()
        self.pretrained_transformer = PRETRAINED_MODELS[pretrained_model][0].from_pretrained(pretrained_model)

        if freeze_pretrained:
            for p in self.pretrained_transformer.parameters():
                p.requires_grad = False

        bert_dim = PRETRAINED_MODELS[pretrained_model][2]

        if lstm_dim == -1:
            hidden_size = bert_dim
        else:
            hidden_size = lstm_dim

        self.hidden_size = hidden_size

        self.lstm = nn.LSTM(input_size=bert_dim,
                            hidden_size=hidden_size,
                            num_layers=1,
                            bidirectional=True)

        self.linear = nn.Linear(in_features=hidden_size * 2,
                                out_features=len(targets))

    def forward(self, x: Tensor, attn_masks: Tensor) -> Tensor:
        # add dummy batch for single sample
        if len(x.shape) == 1:
            x = x.view(1, x.shape[0])
        # (B, N, E) -> (B, N, E)
        x = self.pretrained_transformer(x, attention_mask=attn_masks)[0]
        # (B, N, E) -> (N, B, E)
        x = torch.transpose(x, 0, 1)
        x, (_, _) = self.lstm(x)
        # (N, B, E) -> (B, N, E)
        x = torch.transpose(x, 0, 1)
        x = self.linear(x)
        return x

    def save(self, save_path: Path_type) -> None:
        torch.save(self.state_dict(), save_path)

    def load(self, load_path: Path_type, *args, **kwargs) -> None:
        self.load_state_dict(torch.load(load_path, *args, **kwargs))

    def modify_last_linear(self, *args, **kwargs):
        self.linear = nn.Linear(*args, **kwargs)

In [ ]:
import re
from glob import glob
def get_last_epoch_params(weights_dir: Path) -> Tuple[int, float]:
    """
    Get last epoch number and last best accuracy

    Args:
        weights_dir (Path): directory where wheights are stored

    Returns:
        tuple[int, float]: epoch number, best accuracy
    """
    weights = list(glob(f'{common_path}/weights/*.pt'))
    print (weights)
    last_weight = str(sorted(weights)[-1])
    if (match := re.search(r'_ep(\d+)_(\d+)\.pt', last_weight)):
        epoch = int(match.group(1))
        best_acc = float('0.' + match.group(2))
        return epoch, best_acc
    return 0, 0.0

def log_text(file_path, log):
    if not log.endswith('\n'):
        log += '\n'

    print(log)
    with open(file_path, 'a') as f:
        f.write(log)

def log_train_epoch(file_path, epoch, train_loss, train_accuracy):
    log = f"epoch: {epoch}, Train loss: {train_loss}, Train accuracy: {train_accuracy}\n"
    log_text(file_path, log)

def log_val_epoch(file_path, epoch, val_loss, val_acc):
    log = f"epoch: {epoch}, Val loss: {val_loss}, Val accuracy: {val_acc}\n"
    log_text(file_path, log)


def log_test_metrics(file_path, precision, recall, f1, accuracy, cm):
    log = (f"Precision: {precision}\n"
           f"Recall: {recall}\n"
           f"F1 score: {f1}\n"
           f"Accuracy: {accuracy}\n"
           f"Confusion Matrix:\n{cm}\n")
    log_text(file_path, log)

def save_weights(model: CorrectionModel,
                 weights_dir: Path,
                 epoch: int,
                 accuracy: float) -> None:
    """
    Save weights with epoch number and accuracy

    Args:
        model (CorrectionModel): model instance
        weights_dir (Path): path to store weight
        epoch (int): epoch number
        accuracy (float): accuracy
    """
    acc = str(accuracy)[2:6]
    save_path = f"{common_path}/weights/weights_ep{epoch}_{acc}.pt"
    model.save(save_path)

def log_target_test_metrics(file_path, target, precision, recall, f1):
    log = (f"{target}:\n"
           f"\tPrecision: {round(precision, 4)}\n"
           f"\tRecall: {round(recall, 4)}\n"
           f"\tF1 score: {round(f1, 4)}\n")
    log_text(file_path, log)

#Метрики

In [ ]:
def calc_accuracy_metrics(model: nn.Module,
                          loader: DataLoader
                          ) -> Tuple[ndarray, ndarray, ndarray, float, ndarray]:
    """
    Calculate different accuracy metrics

    Args:
        model (nn.Module): model instance
        loader (DataLoader): data loader

    Returns:
        tuple[ndarray, ndarray, ndarray, float, ndarray]: precision, recall, \
            f1_score, accuracy, confusion_matrx
    """
    num_iteration = 0
    correct = 0
    total = 0

    model.eval()

    # +1 for overall result
    tp = np.zeros(1 + len(TARGETS), dtype=np.int64)
    fp = np.zeros(1 + len(TARGETS), dtype=np.int64)
    fn = np.zeros(1 + len(TARGETS), dtype=np.int64)
    cm = np.zeros((len(TARGETS), len(TARGETS)), dtype=np.int64)

    with torch.no_grad():
        for x, y, att, y_mask in tqdm(loader, desc='test'):
            x = x.to(DEVICE)
            y = y.view(-1).to(DEVICE)
            att = att.to(DEVICE)
            y_mask = y_mask.view(-1).to(DEVICE)

            y_predict = model(x, att)
            y_predict = y_predict.view(-1, y_predict.shape[2])
            y_predict = torch.argmax(y_predict, dim=1).view(-1)

            num_iteration += 1
            y_mask = y_mask.view(-1)
            correct += int(torch.sum(y_mask * (y_predict == y).long()).item())
            total += int(torch.sum(y_mask).item())
            for i in range(y.shape[0]):
                if y_mask[i] == 0:
                    # we can ignore this because we know there won't be
                    # any punctuation in this position since we created
                    # this position due to padding or sub-word tokenization
                    continue
                cor = y[i]
                prd = y_predict[i]
                if cor == prd:
                    tp[cor] += 1
                else:
                    fn[cor] += 1
                    fp[prd] += 1
                cm[cor][prd] += 1

    # ignore first index which is for no punctuation
    tp[-1] = np.sum(tp[1:])
    fp[-1] = np.sum(fp[1:])
    fn[-1] = np.sum(fn[1:])

    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1 = 2 * precision * recall / (precision + recall)
    accuracy = correct / total

    return precision, recall, f1, accuracy, cm

#Датасет

In [ ]:
class BaseDataset(torch.utils.data.Dataset):
    def __init__(self, files: Union[str, List[str]],
                 tokenizer: PreTrainedTokenizer,
                 targets: Dict[str, int],
                 sequence_len: int,
                 token_style: str) -> None:

        if isinstance(files, list):
            self.data = []
            for file in files:
                self.data += self._parse_data(file, tokenizer, targets, sequence_len, token_style)
        else:
            self.data = self._parse_data(files, tokenizer, targets, sequence_len, token_style)

    @classmethod
    def _parse_data(cls,
                    file_path: str,
                    tokenizer: PreTrainedTokenizer,
                    targets: Dict[str, int],
                    seq_len: int,
                    token_style: str):
        """Parse file to train data

        Args:
            file_path (`str`): text file path that contains tokens and punctuations separated by tab in lines
            tokenizer (`PreTrainedTokenizer`): tokenizer that will be used to further tokenize word for BERT like models
            targets (`dict[str, int]`): dictionary of target: label
            seq_len (`int`): maximum length of each sequence
            token_style (`str`): For getting index of special tokens in pretrained.TOKEN_IDX

        Returns:
            list[Batch]: each having sequence_len punctuation_mask is used to ignore special indices like padding and intermediate sub-word token during evaluation
        """
        with open(file_path, 'r', encoding='utf-8') as file:
            x, y = [], []
            for i, line in enumerate(file):
                if (line := line.strip()):
                    token = line.rsplit('\t', 1)
                    if len(token) == 2:
                        x.append(token[0])
                        target = targets[token[1]]
                        y.append(target)
                    else:
                        continue

        data = cls.parse_tokens(x, tokenizer, seq_len, token_style, y)
        # cls._add_targets_to_data(y, data)
        return data

    @classmethod
    def parse_tokens(cls,
                     tokens: Union[List[str], Tuple[str]],
                     tokenizer: PreTrainedTokenizer,
                     seq_len: int,
                     token_style: str,
                     targets: Optional[List[int]] = None,
                     debug: bool = True):
        """
        Convert tokenized data for model prediction

        Args:
            tokens (`Union[list[str], tuple[str]]`): splited tokens
            tokenizer (`PreTrainedTokenizer`): tokenizer which split tokens to subtokens
            seq_len (`int`): sequence length
            token_style (`str`): token_style from pretrained.TOKEN_IDX

        Returns:
            (`list[BatchWithoutTarget]`): list of bathces

        ```txt
        tokens    : [token  token  ##token  PAD ]
             x    : [321    1233   23121    101 ]
             y    : [tar    0      tar      0   ]
        y_mask    : [1      0      1        0   ]
        attn_mask : [1      1      1        0   ]
        ```

        """
        data_items = []
        # loop until end of the entire text
        idx = 0

        if debug:
            pbar = tqdm(total=len(tokens))

        while idx < len(tokens):
            x = [TOKEN_IDX[token_style]['START_SEQ']]
            w_id = [-1]    # word indexes
            y = [0]
            y_mask = [1] if targets else [0]

            # loop until we have required sequence length
            # -1 because we will have a special end of sequence token at the end
            while len(x) < seq_len - 1 and idx < len(tokens):
                word_pieces = tokenizer.tokenize(tokens[idx])

                # if taking these tokens exceeds sequence length we finish
                # current sequence with padding
                # then start next sequence from this token
                if len(word_pieces) + len(x) >= seq_len:
                    break
                for i in range(len(word_pieces) - 1):
                    x.append(tokenizer.convert_tokens_to_ids(word_pieces[i]))
                    w_id.append(idx)
                    y.append(0)
                    y_mask.append(0)
                if len(word_pieces) > 0:
                    x.append(tokenizer.convert_tokens_to_ids(word_pieces[-1]))
                else:
                    x.append(TOKEN_IDX[token_style]['UNK'])

                w_id.append(idx)

                if targets:
                    y.append(targets[idx])
                else:
                    y.append(0)

                y_mask.append(1)

                idx += 1
                if debug:
                    pbar.update(1)

            x.append(TOKEN_IDX[token_style]['END_SEQ'])
            w_id.append(-1)
            y.append(0)
            if targets:
                y_mask.append(1)
            else:
                y_mask.append(0)

            # Fill with pad tokens
            if len(x) < seq_len:
                x = x + [TOKEN_IDX[token_style]['PAD'] for _ in range(seq_len - len(x))]
                w_id = w_id + [-100 for _ in range(seq_len - len(w_id))]
                y = y + [0 for _ in range(seq_len - len(y))]
                y_mask = y_mask + [0 for _ in range(seq_len - len(y_mask))]

            attn_mask = [1 if token != TOKEN_IDX[token_style]['PAD'] else 0 for token in x]

            data_items.append([x, w_id, attn_mask, y, y_mask])

        if debug:
            pbar.close()

        return data_items

    @classmethod
    def _add_targets_to_data(cls,
                             targets: List[int],
                             data: List[List[List[int]]]) -> None:

        targets = targets.copy()

        for idx, batch in tqdm(enumerate(data), total=len(data)):
            y = []
            y_mask = []
            word_ids = batch[1]
            for i, w_id in enumerate(word_ids):
                if w_id == -100:
                    y.append(0)
                    y_mask.append(0)
                elif w_id == -1:
                    y.append(0)
                    y_mask.append(1)
                elif w_id != word_ids[i + 1]:
                    y.append(targets.pop(0))
                    y_mask.append(1)
                else:
                    y.append(0)
                    y_mask.append(0)

            data[idx].extend([y, y_mask])

    def __len__(self) -> int:
        return len(self.data)

    def __getitem__(self, index: int) -> Tuple[Tensor, Tensor, Tensor, Tensor]:
        x = self.data[index][0]
        attn_mask = self.data[index][2]
        y = self.data[index][3]
        y_mask = self.data[index][4]

        x = torch.tensor(x)
        attn_mask = torch.tensor(attn_mask)
        y = torch.tensor(y)
        y_mask = torch.tensor(y_mask)

        return x, y, attn_mask, y_mask

# Код модели

In [ ]:
class RepunctDataset(BaseDataset):
    def __init__(self,
                 files: Union[str, List[str]],
                 tokenizer: PreTrainedTokenizer,
                 targets: Dict[str, int],
                 sequence_len: int,
                 token_style: str,
                 is_train=False,
                 augment_rate=0.,
                 augment_type='substitute') -> None:
        """Preprocess data for restore punctuation

        Args:
            files (`Union[str, list[str]]`): single file or list of text files containing tokens and punctuations separated by tab in lines
            tokenizer (`PreTrainedTokenizer`): tokenizer that will be used to further tokenize word for BERT like models
            targets (`dict[str, int]`): dict with targets
            sequence_len (`int`): length of each sequence
            token_style (`str`): For getting index of special tokens in pretrained.TOKEN_IDX
            is_train (`bool, optional`): if false do not apply augmentation. Defaults to False.
            augment_rate (`float, optional`): percent of data which should be augmented. Defaults to 0.0.
            augment_type (`str, optional`): augmentation type. Defaults to 'substitute'.
        """
        super().__init__(files, tokenizer, targets, sequence_len, token_style)

        self.sequence_len = sequence_len
        self.token_style = token_style
        self.is_train = is_train
        self.augment_type = augment_type
        self.augment_rate = augment_rate

    # TODO: remove method
    @classmethod
    def _parse_data_old(cls,
                        file_path: str,
                        tokenizer: PreTrainedTokenizer,
                        targets: Dict[str, int],
                        seq_len: int,
                        token_style: str):
        """Parse file to train data

        Args:
            file_path (`str`): text file path that contains tokens and punctuations separated by tab in lines
            tokenizer (`PreTrainedTokenizer`): tokenizer that will be used to further tokenize word for BERT like models
            targets (`dict[str, int]`): dictionary of target: label
            seq_len (`int`): maximum length of each sequence
            token_style (`str`): For getting index of special tokens in pretrained.TOKEN_IDX

        Returns:
            list[Batch]: each having sequence_len punctuation_mask is used to ignore special indices like padding and intermediate sub-word token during evaluation
        """
        data_items = []
        with open(file_path, 'r', encoding='utf-8') as f:
            lines = [line for line in f.read().split('\n') if line.strip()]
            idx = 0
            # loop until end of the entire text
            while idx < len(lines):
                x = [TOKEN_IDX[token_style]['START_SEQ']]
                y = [0]
                y_mask = [1]  # which positions we need to consider while evaluating i.e., ignore pad or sub tokens

                # loop until we have required sequence length
                # -1 because we will have a special end of sequence token at the end
                while len(x) < seq_len - 1 and idx < len(lines):
                    word, target = lines[idx].rsplit('\t', 1)
                    tokens = tokenizer.tokenize(word)

                    # if taking these tokens exceeds sequence length we finish current sequence with padding
                    # then start next sequence from this token
                    if len(tokens) + len(x) >= seq_len:
                        break
                    else:
                        for i in range(len(tokens) - 1):
                            x.append(tokenizer.convert_tokens_to_ids(tokens[i]))
                            y.append(0)
                            y_mask.append(0)
                        if len(tokens) > 0:
                            x.append(tokenizer.convert_tokens_to_ids(tokens[-1]))
                        else:
                            x.append(TOKEN_IDX[token_style]['UNK'])
                        y.append(targets[target])
                        y_mask.append(1)
                        idx += 1

                x.append(TOKEN_IDX[token_style]['END_SEQ'])
                y.append(0)
                y_mask.append(1)

                # Fill with pad tokens
                if len(x) < seq_len:
                    x = x + [TOKEN_IDX[token_style]['PAD'] for _ in range(seq_len - len(x))]
                    y = y + [0 for _ in range(seq_len - len(y))]
                    y_mask = y_mask + [0 for _ in range(seq_len - len(y_mask))]

                attn_mask = [1 if token != TOKEN_IDX[token_style]['PAD'] else 0 for token in x]
                data_items.append([x, y, attn_mask, y_mask])

        return data_items

    def _augment(self, x, y, y_mask):
        x_aug = []
        y_aug = []
        y_mask_aug = []
        for i in range(len(x)):
            r = np.random.rand()
            if r < self.augment_rate:
                AUGMENTATIONS[self.augment_type](x, y, y_mask, x_aug, y_aug, y_mask_aug, i, self.token_style)
            else:
                x_aug.append(x[i])
                y_aug.append(y[i])
                y_mask_aug.append(y_mask[i])

        if len(x_aug) > self.sequence_len:
            # len increased due to insert
            x_aug = x_aug[:self.sequence_len]
            y_aug = y_aug[:self.sequence_len]
            y_mask_aug = y_mask_aug[:self.sequence_len]
        elif len(x_aug) < self.sequence_len:
            # len decreased due to delete
            x_aug = x_aug + [TOKEN_IDX[self.token_style]['PAD'] for _ in range(self.sequence_len - len(x_aug))]
            y_aug = y_aug + [0 for _ in range(self.sequence_len - len(y_aug))]
            y_mask_aug = y_mask_aug + [0 for _ in range(self.sequence_len - len(y_mask_aug))]

        attn_mask = [1 if token != TOKEN_IDX[self.token_style]['PAD'] else 0 for token in x]
        return x_aug, y_aug, attn_mask, y_mask_aug

    def __getitem__(self, index: int) -> Tuple[Tensor, Tensor, Tensor, Tensor]:
        x = self.data[index][0]
        attn_mask = self.data[index][2]
        y = self.data[index][3]
        y_mask = self.data[index][4]

        if self.is_train and self.augment_rate > 0:
            x, y, attn_mask, y_mask = self._augment(x, y, y_mask)

        x = torch.tensor(x)
        attn_mask = torch.tensor(attn_mask)
        y = torch.tensor(y)
        y_mask = torch.tensor(y_mask)

        return x, y, attn_mask, y_mask

# Модель

In [ ]:
all_punct = ["'", '"',
              ".", ",", "!", "?",
              "-", ":", ";"]
names_punct = [
                'S_QUOTE', 'D_QUOTE',
                'PERIOD', 'COMMA', 'EX_MARK', 'Q_MARK',
                'DASH', 'COLUMN', 'SEMI_COL'
                ]
punctuation_marks = dict(zip(all_punct, names_punct))
punctuation_marks[' '] = 'O'

# targets for DeepPavlov/rubert-base-cased-sentence:
# targets = {'O': 102, 'S_QUOTE': 118, 'D_QUOTE': 108, 'PERIOD': 132, 'COMMA': 128, 'EX_MARK': 106, 'Q_MARK': 166, 'DASH': 130, 'COLUMN': 156, 'SEMI_COL': 158}
targets = {'O': 0, 'S_QUOTE': 1, 'D_QUOTE': 2, 'PERIOD': 3, 'COMMA': 4, 'EX_MARK': 5, 'Q_MARK': 6, 'DASH': 7, 'COLUMN': 8, 'SEMI_COL': 9}


weights = [1.5] * len(targets)

In [ ]:
# MODEL
print('Loading model...')

# targets = {'O': 0, 'M': 1} # 'TARGETS'
# weights = [1.5, 1.5] # weights for targets'
lr = 5e-6
decay = 0
cuda = False
resume = True
fine_tune = True
lstm_dim = -1
pretrained_model = 'DeepPavlov/rubert-base-cased-sentence'
freeze_pretrained = False
model_save_path = f'{common_save_path}model_saved'
alpha_sub = 0.40
# probability of applying delete operation on tokens selected for augmentation
alpha_del = 0.40
tokenizer = None
# substitution strategy: 'unk' -> replace with unknown tokens, 'rand' -> replace with random tokens from vocabulary
sub_style = 'unk'

labml_flag = False
sequence_length = 256

TARGETS = targets
DEVICE = torch.device('cuda' if (cuda and torch.cuda.is_available()) else 'cpu')

MODEL = CorrectionModel(pretrained_model,
                        targets=TARGETS,
                        freeze_pretrained=freeze_pretrained,
                        lstm_dim=lstm_dim)


MODEL.to(DEVICE)

Loading model...


CorrectionModel(
  (pretrained_transformer): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1

In [ ]:
WEIGHTS = torch.FloatTensor(np.array(weights)).to(DEVICE)
CRITERION = nn.CrossEntropyLoss(weight=WEIGHTS)
OPTIMIZER = torch.optim.Adam(MODEL.parameters(), lr=lr, weight_decay=decay)
print('Model was loaded.')

# TOKENIZER
print('Loading tokenizer...')
tokenizer = PRETRAINED_MODELS[pretrained_model][1].from_pretrained(pretrained_model)
token_style = PRETRAINED_MODELS[pretrained_model][3]
SEQ_LEN = sequence_length
print('Tokenizer was loaded.')

# CONFIG AUGMENTATION
AUG_RATE = 0.
AUG_TYPE = 'all'


# DATASETS
data_loader_params = {
    'batch_size': 4,
    'shuffle': True,
    'num_workers': 1
}

Model was loaded.
Loading tokenizer...
Tokenizer was loaded.


In [ ]:
# file_paths = ['file1.txt', 'file2.txt', 'file3.txt']
# file_paths = [os.path.join(files, file) for file in os.listdir(files)]:
# 1734

train_flist = []
val_flist = []
test_flist = []
# r=root, d=directories, f = files
for r, d, f in os.walk(f'{common_path}/literature_csv'):
    i = 0
    for file in f:
        if i <= 1000:
            if file.endswith(('.txt')):
                train_flist.append(os.path.join(r, file))
        elif 1000 < i <= 1300:
            if file.endswith(('.txt')):
                val_flist.append(os.path.join(r, file))
        elif 1300 < i <= 1734:
            if file.endswith(('.txt')):
                test_flist.append(os.path.join(r, file))
        i += 1
print(test_flist)

['/content/drive/MyDrive/Data_punct/Datasets/literature_csv/000451.txt', '/content/drive/MyDrive/Data_punct/Datasets/literature_csv/000452.txt', '/content/drive/MyDrive/Data_punct/Datasets/literature_csv/000453.txt', '/content/drive/MyDrive/Data_punct/Datasets/literature_csv/000454.txt', '/content/drive/MyDrive/Data_punct/Datasets/literature_csv/000455.txt', '/content/drive/MyDrive/Data_punct/Datasets/literature_csv/000456.txt', '/content/drive/MyDrive/Data_punct/Datasets/literature_csv/000457.txt', '/content/drive/MyDrive/Data_punct/Datasets/literature_csv/000458.txt', '/content/drive/MyDrive/Data_punct/Datasets/literature_csv/000459.txt', '/content/drive/MyDrive/Data_punct/Datasets/literature_csv/000460.txt', '/content/drive/MyDrive/Data_punct/Datasets/literature_csv/000461.txt', '/content/drive/MyDrive/Data_punct/Datasets/literature_csv/000462.txt', '/content/drive/MyDrive/Data_punct/Datasets/literature_csv/000463.txt', '/content/drive/MyDrive/Data_punct/Datasets/literature_csv/0004

In [ ]:
print('Loading train data...')
# f'{common_path}/train.txt'
train_dataset = RepunctDataset(train_flist, tokenizer=tokenizer, targets=TARGETS,
                               sequence_len=SEQ_LEN, token_style=token_style,
                               is_train=True, augment_rate=AUG_RATE,
                               augment_type=AUG_TYPE)
train_loader = DataLoader(train_dataset, **data_loader_params)

Loading train data...


100%|██████████| 383/383 [00:00<00:00, 15534.61it/s]


In [ ]:
print('Loading validation data...')
# f'{common_path}/valid.txt'
val_dataset = RepunctDataset(val_flist, tokenizer=tokenizer, targets=TARGETS,
                             sequence_len=SEQ_LEN, token_style=token_style,
                             is_train=True)
val_loader = DataLoader(val_dataset, **data_loader_params)

Loading validation data...


100%|██████████| 210/210 [00:00<00:00, 20593.96it/s]


In [ ]:
print('Loading test data...')
# f'{common_path}/test.txt'
test_dataset = RepunctDataset(test_flist, tokenizer=tokenizer, targets=TARGETS,
                                      sequence_len=SEQ_LEN, token_style=token_style,
                                      is_train=True, augment_rate=AUG_RATE,
                                      augment_type=AUG_TYPE)
test_loader = DataLoader(test_dataset, **data_loader_params)

print('Data was loaded.')

Loading test data...


100%|██████████| 444/444 [00:00<00:00, 21394.59it/s]

Data was loaded.


Выдаются 4 кусочка по 256

In [ ]:
len(next(iter(test_loader)))

4

In [ ]:
next(iter(test_loader))[0].shape

torch.Size([4, 256])

In [ ]:
LOG_PATH = f"{common_path}/debug/logs.txt"
WEIGHTS_SAVE_DIR = f'{common_path}/weights'

#Обучение

In [ ]:
gradient_clip =-1 #отключён
def train_epoch(model: nn.Module,
                loader: DataLoader,
                criterion: nn.Module,
                optimizer: torch.optim.Optimizer
                ) -> Tuple[float, float]:
    """
    Train single epoch

    Args:
        model (nn.Module): model instance
        loader (DataLoader): data loader
        criterion (nn.Module): criterion
        optimizer (torch.optim.Optimizer): optimizer

    Returns:
        tuple[float, float]: train_loss, train_accuracy
    """
    train_loss = 0.0
    train_iteration = 0
    correct = 0.
    total = 0.

    model.train()
    for x, y, att, y_mask in tqdm(loader, desc='train'):
        # print ('x:',x.shape)
        # print ('y:',y.shape)
        # print ('y_mask:',y_mask.shape)
        # print ('att:',att.shape)

        x = x.to(DEVICE)
        y = y.view(-1).to(DEVICE)
        att = att.to(DEVICE)
        y_mask = y_mask.view(-1).to(DEVICE)

        print ('\nProgress1')
        y_predict = model(x, att)
        print ('\nProgress111')

        print (y_predict.shape)
        # print (y_predict)

        y_predict = y_predict.view(-1, y_predict.shape[2])
        print ('y_predict.shape: ', y_predict.shape)
        print ('Progress222')

        loss = criterion(y_predict, y)####
        print ('Progress2')

        y_predict = torch.argmax(y_predict, dim=1).view(-1)
        correct += torch.sum(y_mask * (y_predict == y).long()).item()
        print ('Progress3')

        optimizer.zero_grad()
        train_loss += loss.item()
        train_iteration += 1
        loss.backward()
        print ('Progress4')

        if gradient_clip > 0:
            torch.nn.utils.clip_grad_norm_(model.parameters(), gradient_clip)

        optimizer.step()
        total += torch.sum(y_mask.view(-1)).item()

    train_loss /= train_iteration
    train_accuracy = correct / total

    return train_loss, train_accuracy


def validate_epoch(model: nn.Module,
                   loader: DataLoader,
                   criterion: nn.Module) -> Tuple[float, float]:
    """
    Validate sinlge epoch

    Args:
        model (nn.Module): model instance
        loader (DataLoader): data loader (should be different from train loader)
        criterion (nn.Module): criterion

    Returns:
        tuple[float, float]: validation_loss, validation_accuracy
    """
    num_iteration = 0
    correct = 0.
    total = 0.
    val_loss = 0.0

    model.eval()
    with torch.no_grad():
        for x, y, att, y_mask in tqdm(loader, desc='eval'):
            x = x.to(DEVICE)
            y = y.view(-1).to(DEVICE)
            att = att.to(DEVICE)
            y_mask = y_mask.view(-1).to(DEVICE)

            y_predict = model(x, att)
            y_predict = y_predict.view(-1, y_predict.shape[2])

            loss = criterion(y_predict, y)
            y_predict = torch.argmax(y_predict, dim=1).view(-1)

            val_loss += loss.item()
            num_iteration += 1
            y_mask = y_mask.view(-1)
            correct += torch.sum(y_mask * (y_predict == y).long()).item()
            total += torch.sum(y_mask).item()

    val_loss = val_loss / num_iteration
    val_acc = correct / total

    return val_loss, val_acc

In [ ]:
def train() -> None:
    epoch = 2
    """Train: global function"""
    resume = False
    if not resume:
        best_val_acc = 0.0
        epochs = range(epoch)
    #else:
    #    last_epoch, best_val_acc = get_last_epoch_params('/content/drive/My Drive/debug_data/weights')
    #    epochs = range(last_epoch + 1, last_epoch + 1 + epoch)

    for epoch in epochs:
        train_loss, train_acc = train_epoch(MODEL, train_loader, CRITERION, OPTIMIZER)
        log_train_epoch(LOG_PATH, epoch, train_loss, train_acc)

        val_loss, val_acc = validate_epoch(MODEL, val_loader, CRITERION)
        log_val_epoch(LOG_PATH, epoch, val_loss, val_acc)

        tracker.save(epoch, {'train_loss': train_loss,
                              'train_accuracy': train_acc,
                              'val_loss': val_loss,
                              'val_accuracy': val_acc})

        # store_every_weight = True ###########
        store_best_weights = True
        # if store_every_weight:
        #     save_weights(MODEL, WEIGHTS_SAVE_DIR, epoch, val_acc)
        # elif store_best_weights and (val_acc > best_val_acc):
        #     best_val_acc = val_acc
        #     save_weights(MODEL, WEIGHTS_SAVE_DIR, epoch, val_acc)
        best_val_acc = val_acc
        save_weights(MODEL, WEIGHTS_SAVE_DIR, epoch, val_acc)

    log_text(LOG_PATH, f"Best validation Acc: {best_val_acc}")


def test():
    """Test: global function"""
    # precision, recall, f1, accuracy, cm = calc_accuracy_metrics(MODEL, test_loader)
    precision, recall, f1, accuracy, cm = calc_accuracy_metrics(MODEL, val_loader)
    log_test_metrics(LOG_PATH, precision, recall, f1, accuracy, cm)

    non_O_keys = " + ".join(list(TARGETS)[1:])
    targets = list(TARGETS) + [non_O_keys]
    for i, target in enumerate(targets):
        log_target_test_metrics(LOG_PATH, target, precision[i], recall[i], f1[i])

#Train

In [ ]:
train()

train:   0%|          | 0/462 [00:00<?, ?it/s]


Progress1

Progress111
torch.Size([4, 256, 10])
y_predict.shape:  torch.Size([1024, 10])
Progress222
Progress2
Progress3
Progress4


train:   0%|          | 1/462 [00:05<41:32,  5.41s/it]


Progress1

Progress111
torch.Size([4, 256, 10])
y_predict.shape:  torch.Size([1024, 10])
Progress222
Progress2
Progress3
Progress4


train:   0%|          | 2/462 [00:08<30:22,  3.96s/it]

...

train: 100%|██████████| 462/462 [26:02<00:00,  3.38s/it]


epoch: 1, Train loss: 0.250714235797976, Train accuracy: 0.8654603791293339



eval: 100%|██████████| 139/139 [02:18<00:00,  1.00it/s]

epoch: 1, Val loss: 0.17774553779217836, Val accuracy: 0.9063121492722598



Best validation Acc: 0.9063121492722598



In [ ]:
debug_other = f'{common_path}/Аннушка.txt'

In [ ]:
tokenizer = PRETRAINED_MODELS[pretrained_model][1].from_pretrained(pretrained_model)
token_style = PRETRAINED_MODELS[pretrained_model][3]
SEQ_LEN = sequence_length
# CONFIG AUGMENTATION
AUG_RATE = 0.
AUG_TYPE = 'all'
# DATASETS
data_loader_params = {
    'batch_size': 4,
    'shuffle': True,
    'num_workers': 1
}

In [ ]:
#  x, y, att, and y_mask are the input features, target labels, attention masks,
#  and target masks, respectively, for each batch of data in the dataloader.
import pandas as pd
pd.set_option('display.max_rows', None)

targets_inversed = {v: k for k, v in TARGETS.items()}

test2_dataset = RepunctDataset(debug_other, tokenizer=tokenizer, targets=TARGETS, #f'{common_path}/debug_sentences.txt'
                                      sequence_len=SEQ_LEN, token_style=token_style,
                                      is_train=False, augment_rate=AUG_RATE, #istrainwastrue
                                      augment_type=AUG_TYPE)
test2_loader = DataLoader(test2_dataset, **data_loader_params)


def predict_one(model: nn.Module,
                loader: DataLoader,
                criterion: nn.Module) -> Tuple[float, float]:

    num_iteration = 0
    correct = 0.
    total = 0.
    val_loss = 0.0

    model.eval()
    with torch.no_grad():
        for x, y, att, y_mask in tqdm(test2_loader, desc='eval'):
            x = x.to(DEVICE)
            y = y.view(-1).to(DEVICE)
            att = att.to(DEVICE)
            y_mask = y_mask.view(-1).to(DEVICE)

            y_predict = model(x, att)
            y_predict = y_predict.view(-1, y_predict.shape[2])

            loss = criterion(y_predict, y)
            y_predict = torch.argmax(y_predict, dim=1).view(-1)

            val_loss += loss.item()
            num_iteration += 1
            y_mask = y_mask.view(-1)

            non_O_keys = " + ".join(list(TARGETS)[1:])
            # targets = list(TARGETS) + [non_O_keys]
            # for i, target in enumerate(targets):
            preds_labels = []
            tokens = []
            # for i in range(x.shape[0]):
            #     tokens.append(tokenizer.decode(x[i]))

            for i in range(x.shape[0]):
                token_ids = x[i].tolist()
                token_subwords = tokenizer.convert_ids_to_tokens(token_ids)
                tokens.append(token_subwords)
            for i in y_predict.data:
                preds_labels.append(targets_inversed[i.item()])

            y_true_label = []
            for i in y.tolist():
                y_true_label.append(targets_inversed[i])

            return {'tokens': tokens[0],
                    'y_true': y.tolist(),
                    'y_true_label': y_true_label,
                    'y_predict': y_predict,
                    'preds_labels': preds_labels}

sent_table = predict_one(MODEL, test2_loader, CRITERION)
sent_table = pd.DataFrame.from_dict(sent_table)
sent_table



100%|██████████| 17/17 [00:00<00:00, 5178.15it/s]

eval:   0%|          | 0/1 [00:00<?, ?it/s]


,tokens,y_true,y_true_label,y_predict,preds_labels
0,[CLS],0,O,0,O
1,Анну,0,O,0,O
2,##шка,0,O,0,O
3,уже,0,O,0,O
4,купила,0,O,0,O
5,подсолн,0,O,0,O
6,##ечное,0,O,0,O
7,масло,4,COMMA,0,O
8,и,0,O,0,O
9,не,0,O,0,O
